In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/gender_submission.csv
/kaggle/input/titanic/test.csv


In [2]:
import sys
import matplotlib 
import scipy as sp 
import IPython
from IPython import display 
import sklearn 
from sklearn.preprocessing import MinMaxScaler

In [3]:
from sklearn import svm, tree, linear_model, neighbors, naive_bayes, ensemble, discriminant_analysis, gaussian_process
from xgboost import XGBClassifier

from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn import feature_selection
from sklearn import model_selection
from sklearn import metrics

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import seaborn as sns

%matplotlib inline
mpl.style.use('ggplot')
sns.set_style('white')
pylab.rcParams['figure.figsize'] = 12,8

In [4]:
train_data = pd.read_csv("/kaggle/input/titanic/train.csv")
test_data = pd.read_csv("/kaggle/input/titanic/test.csv")

test_data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [5]:
data = [train_data, test_data]

for dataset in data:
    dataset['Age'].fillna(dataset['Age'].mean(), inplace = True)
    dataset['Embarked'].fillna(dataset['Embarked'].mode()[0], inplace = True)
    dataset['Fare'].fillna(dataset['Fare'].median(), inplace = True)

pass
columns_to_drop = ["PassengerId", "Cabin", "Ticket"]
test_data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [6]:
for dataset in data:
    dataset["Family_size"] = dataset ["SibSp"] + dataset['Parch'] + 1
    dataset['IsAlone'] = 1
    dataset['IsAlone'].loc[dataset['Family_size'] > 1] = 0
    dataset['Title'] = dataset['Name'].str.split(", ", expand = True)[1].str.split(".", expand = True)[0]
    dataset['AgeBin'] = pd.cut(dataset['Age'].astype(int), 5)
    dataset['FareBin'] = pd.cut(dataset['Fare'].astype(int), 4)

stat_min = 10
train_title_names = (train_data['Title'].value_counts() < stat_min)
test_title_names = (test_data['Title'].value_counts() < stat_min)

train_data['Title'] = train_data['Title'].apply(lambda x: 'Misc' if train_title_names.loc[x] == True else x)
test_data['Title'] = test_data['Title'].apply(lambda x: 'Misc' if test_title_names.loc[x] == True else x)


train_data.head()

/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Family_size,IsAlone,Title,AgeBin,FareBin
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,2,0,Mr,"(16.0, 32.0]","(-0.512, 128.0]"
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,2,0,Mrs,"(32.0, 48.0]","(-0.512, 128.0]"
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,1,1,Miss,"(16.0, 32.0]","(-0.512, 128.0]"
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,2,0,Mrs,"(32.0, 48.0]","(-0.512, 128.0]"
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,1,1,Mr,"(32.0, 48.0]","(-0.512, 128.0]"


In [7]:
label = LabelEncoder()

for dataset in data: 
    dataset['Title_Code'] = label.fit_transform(dataset['Title'])
    dataset['Sex_Code'] = label.fit_transform(dataset['Sex'])
    dataset['Embarked_Code'] = label.fit_transform(dataset['Embarked'])
    dataset['AgeBin_Code'] = label.fit_transform(dataset['AgeBin'])
    dataset['FareBin_Code'] = label.fit_transform(dataset['FareBin'])

pass

#train_y = ['Survived']
#train_X = ['Sex', 'Pclass', 'Embarked_Code', 'Title_Code', 'SibSp', 'Parch', 'AgeBin_Code', 'FareBin_Code', 'IsAlone']
#test_X = ['Sex', 'Pclass', 'Embarked_Code', 'Title_Code', 'SibSp', 'Parch', 'AgeBin_Code', 'FareBin_Code', 'IsAlone']

In [8]:
Target = ['Survived']
data1_x = ['Sex','Pclass', 'Embarked', 'Title','SibSp', 'Parch', 'Age', 'Fare', 'IsAlone']
data1_x_calc = ['Sex_Code','Pclass', 'Embarked_Code', 'Title_Code','SibSp', 'Parch', 'Age', 'Fare']
data1_xy =  Target + data1_x


data1_x_bin = ['Sex_Code','Pclass', 'Embarked_Code', 'Title_Code', 'AgeBin_Code', 'FareBin_Code']
data1_xy_bin = Target + data1_x_bin



#define x and y variables for dummy features original
data1_dummy = pd.get_dummies(train_data[data1_x])
data2_dummy = pd.get_dummies(test_data[data1_x])
data1_x_dummy = data1_dummy.columns.tolist()
data1_xy_dummy = Target + data1_x_dummy


data1_dummy.insert(16, 'Survived', train_data['Survived'])
data1_dummy.insert(0, 'PassengerId', train_data['PassengerId'])
data2_dummy.insert(0, 'PassengerId', test_data['PassengerId'])
train_data = data1_dummy
test_data = data2_dummy

test_data.head()

,PassengerId,Pclass,SibSp,Parch,Age,Fare,IsAlone,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S,Title_Master,Title_Misc,Title_Miss,Title_Mr,Title_Mrs
0,892,3,0,0,34.5,7.8292,1,0,1,0,1,0,0,0,0,1,0
1,893,3,1,0,47.0,7.0000,0,1,0,0,0,1,0,0,0,0,1
2,894,2,0,0,62.0,9.6875,1,0,1,0,1,0,0,0,0,1,0
3,895,3,0,0,27.0,8.6625,1,0,1,0,0,1,0,0,0,1,0
4,896,3,1,1,22.0,12.2875,0,1,0,0,0,1,0,0,0,0,1


In [9]:
"""temp = train_data.iloc[0].copy()
train_data.iloc[0] = train_data.iloc[1]
train_data.iloc[1] = temp

temp1 = train_data.iloc[0].copy()
train_data.iloc[0] = train_data.iloc[5]
train_data.iloc[5] = temp1

train_data["Embarked"] = pd.factorize(train_data.Embarked)[0]
test_data["Embarked"] = pd.factorize(test_data.Embarked)[0]

train_data["Cabin"].fillna("Missing", inplace = True)
test_data["Cabin"].fillna("Missing", inplace = True)

#train_data.head()
#test_data.head()

train_data["Cabin"] = train_data["Cabin"].astype(str).str[0]
test_data["Cabin"] = test_data["Cabin"].astype(str).str[0]


train_data.head()"""

'temp = train_data.iloc[0].copy()\ntrain_data.iloc[0] = train_data.iloc[1]\ntrain_data.iloc[1] = temp\n\ntemp1 = train_data.iloc[0].copy()\ntrain_data.iloc[0] = train_data.iloc[5]\ntrain_data.iloc[5] = temp1\n\ntrain_data["Embarked"] = pd.factorize(train_data.Embarked)[0]\ntest_data["Embarked"] = pd.factorize(test_data.Embarked)[0]\n\ntrain_data["Cabin"].fillna("Missing", inplace = True)\ntest_data["Cabin"].fillna("Missing", inplace = True)\n\n#train_data.head()\n#test_data.head()\n\ntrain_data["Cabin"] = train_data["Cabin"].astype(str).str[0]\ntest_data["Cabin"] = test_data["Cabin"].astype(str).str[0]\n\n\ntrain_data.head()'

In [10]:
train_data.head()

,PassengerId,Pclass,SibSp,Parch,Age,Fare,IsAlone,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S,Title_Master,Title_Misc,Title_Miss,Title_Mr,Title_Mrs,Survived
0,1,3,1,0,22.0,7.2500,0,0,1,0,0,1,0,0,0,1,0,0
1,2,1,1,0,38.0,71.2833,0,1,0,1,0,0,0,0,0,0,1,1
2,3,3,0,0,26.0,7.9250,1,1,0,0,0,1,0,0,1,0,0,1
3,4,1,1,0,35.0,53.1000,0,1,0,0,0,1,0,0,0,0,1,1
4,5,3,0,0,35.0,8.0500,1,0,1,0,0,1,0,0,0,1,0,0


In [11]:
from sklearn.impute import SimpleImputer

my_imputer = SimpleImputer()

Titanic_features = ["Pclass", "SibSp", "Parch", "Age","Fare","IsAlone","Sex_female","Sex_male","Embarked_C","Embarked_Q","Embarked_S","Title_Master","Title_Misc","Title_Miss","Title_Mr","Title_Mrs"]
Titanic_prediction_target = ['Survived']

train_X = train_data[Titanic_features]
test_X = test_data[Titanic_features]
train_y = train_data[Titanic_prediction_target]

test_X.head()

,Pclass,SibSp,Parch,Age,Fare,IsAlone,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S,Title_Master,Title_Misc,Title_Miss,Title_Mr,Title_Mrs
0,3,0,0,34.5,7.8292,1,0,1,0,1,0,0,0,0,1,0
1,3,1,0,47.0,7.0000,0,1,0,0,0,1,0,0,0,0,1
2,2,0,0,62.0,9.6875,1,0,1,0,1,0,0,0,0,1,0
3,3,0,0,27.0,8.6625,1,0,1,0,0,1,0,0,0,1,0
4,3,1,1,22.0,12.2875,0,1,0,0,0,1,0,0,0,0,1


In [12]:
"""from sklearn.linear_model import LogisticRegression
my_model = LogisticRegression(max_iter = 1000)

my_model.fit(train_X, train_y)
my_model.predict(train_X)
my_model_predictions = my_model.predict(test_X)

output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': my_model_predictions})

print(output)
output.to_csv("my_submission.csv", index = False)

print("Your submission was successfully saved!")"""

'from sklearn.linear_model import LogisticRegression\nmy_model = LogisticRegression(max_iter = 1000)\n\nmy_model.fit(train_X, train_y)\nmy_model.predict(train_X)\nmy_model_predictions = my_model.predict(test_X)\n\noutput = pd.DataFrame({\'PassengerId\': test_data.PassengerId, \'Survived\': my_model_predictions})\n\nprint(output)\noutput.to_csv("my_submission.csv", index = False)\n\nprint("Your submission was successfully saved!")'

In [13]:
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn import tree
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC


In [14]:
"""import xgboost as xgb
my_model = XGBClassifier()

my_model.fit(train_X, train_y)
my_model.predict(train_X)
my_model_predictions = my_model.predict(test_X)

output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': my_model_predictions})

print(output)
output.to_csv("my_submission.csv", index = False)

print("Your submission was successfully saved!")
"""

'import xgboost as xgb\nmy_model = XGBClassifier()\n\nmy_model.fit(train_X, train_y)\nmy_model.predict(train_X)\nmy_model_predictions = my_model.predict(test_X)\n\noutput = pd.DataFrame({\'PassengerId\': test_data.PassengerId, \'Survived\': my_model_predictions})\n\nprint(output)\noutput.to_csv("my_submission.csv", index = False)\n\nprint("Your submission was successfully saved!")\n'

In [15]:
from sklearn.ensemble import RandomForestRegressor
my_model = RandomForestClassifier(max_depth = 5, min_samples_split = 4, n_estimators = 500, random_state = 20, n_jobs = -1)

my_model.fit(train_X, train_y,)
my_model.predict(train_X)
my_model_predictions = my_model.predict(test_X)

output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': my_model_predictions})

print(output)
output.to_csv("my_submission.csv", index = False)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  after removing the cwd from sys.path.


     PassengerId  Survived
0            892         0
1            893         1
2            894         0
3            895         0
4            896         1
..           ...       ...
413         1305         0
414         1306         1
415         1307         0
416         1308         0
417         1309         1

[418 rows x 2 columns]
